# Assignment 7: Optimising Portfolio Weights
The assignment requires us to generate a portfolio of 10-15 stocks, optimise it according to the given tasks and compare it to the market portfolio.  
The portfolio of stocks should be Long only, i.e. the portfolio only buys an asset and doesn't take any short positions.  

The given tasks are:
> **Task 1:** Optimise your portfolio weights so that the expected return on your 10-15 asset portfolio is equal to the annualised expected return on the market portfolio.
What is the risk of your portfolio now? How does it compare to the market portfolio?
How does it compare to the performance using the initialised weights?

> **Task 2:** Optimise your portfolio weights so that the risk of your 10-15 asset portfolio is equal to the annualised risk of the market portfolio.
What is the expected return on your portfolio now? How does it compare to the market portfolio?

> **Task 3:** Optimise your portfolio weights so that the risk of your portfolio is minimised.
What is the expected return on your portfolio now? How does it compare to the market portfolio?

> **Task 4:** Based on the results from your optimisations, would you invest in your portfolio, or would you invest in the market portfolio instead?
Why?

I decided to generate a portfolio of 10 stocks from the 'Hospitality sector' in India. Stocks are listed on the National Stock Exchange of India, Mumbai (except for '*Sayaji Hotels*' which is listed on the BSE, Mumbai). The companies in the portfolio required to have at least 5 years of price data (i.e. companies that were listed since August, 2017) The companies I chose are:

1. Indian Hotels Company Limited (INDHOTEL)
2. Oriental Hotels Limited (ORIENTHOT)
3. Sayaji Hotels Limited (SAYAJIHOTL)
4. Jubilant FoodWorks Limited (JUBLFOOD)
5. EIH Limited (EIHOTEL)
6. Speciality Restaurants Limited (SPECIALITY)
7. Royal Orchid Hotels Limited (ROHLTD)
8. Wonderla Holidays Limited (WONDERLA)
9. HLV Limited (HLVLTD)
10. BLS International Services Limited (BLS)
  
Price data for these stocks were obtained from *Yahoo Finance*. The price data pertains to the period from 5th August, 2017 to 4th August, 2022 (a 5-year period).

We will be using 'NIFTY MICROCAP 250', a benchmark index of 250 Indian companies listed on the NSE India, which are beyond the NIFTY 500 list of companies (Comapnies ranked *501<sup>st</sup> to 750<sup>th</sup>* as per market capitalisation on NSE), as a representation for the market these portfolio stocks represent. Data for MICROCAP 250 was obtained from *NSE Indices* website.

The reason for choosing MICROCAP 250 instead of the usual 'NIFTY 50' from the NSE is because none of the stocks in the portfolio are in the NIFTY 50 basket of stocks. Also, more than half of the stocks in the portfolio would be considered as '*Small-Caps*' in market parlence.

In [1]:
# We start by importing the important libraries in Python

import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
# Importing price data of the assets in the portfolio

indhotel = pd.read_csv('INDHOTEL.NS.csv')
orienthot = pd.read_csv('ORIENTHOT.NS.csv')
sayajihotl = pd.read_csv('SAYAJIHOTL.BO.csv')
jublfood = pd.read_csv('JUBLFOOD.NS.csv')
eihotel = pd.read_csv('EIHOTEL.NS.csv')
speciality = pd.read_csv('SPECIALITY.NS.csv')
rohltd = pd.read_csv('ROHLTD.NS.csv')
wonderla = pd.read_csv('WONDERLA.NS.csv')
hlvltd = pd.read_csv('HLVLTD.NS.csv')
bls = pd.read_csv('BLS.NS.csv')

In [3]:
# Creating a dataframe with the price data

df = indhotel['Date']
df = df.to_frame()
df['INDHOTEL'] = indhotel['Adj Close']
df['ORIENTHOT'] = orienthot['Adj Close']
df['SAYAJIHOTL'] = sayajihotl['Adj Close']
df['JUBLFOOD'] = jublfood['Adj Close']
df['EIHOTEL'] = eihotel['Adj Close']
df['SPECIALITY'] = speciality['Adj Close']
df['ROHLTD'] = rohltd['Adj Close']
df['WONDERLA'] = wonderla['Adj Close']
df['HLVLTD'] = hlvltd['Adj Close']
df['BLS'] = bls['Adj Close']
df.rename(columns = {'Date' : 'date'}, inplace = True)
df.set_index('date', inplace = True)
df

,INDHOTEL,ORIENTHOT,SAYAJIHOTL,JUBLFOOD,EIHOTEL,SPECIALITY,ROHLTD,WONDERLA,HLVLTD,BLS
date,,,,,,,,,,
2017-08-07,118.398842,34.819897,248.500000,124.582672,128.122437,118.650002,110.201439,340.265076,23.750000,89.706154
2017-08-08,119.224564,33.940849,248.500000,131.109573,129.979980,124.650002,108.611290,336.689545,21.950001,85.210144
2017-08-09,115.738197,33.012974,248.000000,127.996368,128.366852,120.800003,105.479202,338.061005,21.450001,87.517609
2017-08-10,114.453751,32.964138,248.000000,129.843628,128.317978,117.150002,101.913437,333.505829,20.700001,85.257706
2017-08-11,112.604057,32.036255,248.000000,127.740234,128.415741,112.449997,101.865250,330.518066,20.549999,85.543190
...,...,...,...,...,...,...,...,...,...,...
2022-07-29,262.850006,62.900002,232.000000,550.250000,151.149994,186.800003,169.100006,240.449997,9.350000,241.600006
2022-08-01,270.450012,63.400002,236.550003,559.650024,149.399994,188.500000,173.850006,245.800003,9.400000,240.000000
2022-08-02,268.450012,66.050003,247.050003,565.900024,153.899994,188.100006,171.600006,238.250000,9.800000,241.100006


In [4]:
# Calculating daily returns (or percentage change) of the stocks

df_returns = df.pct_change(1).dropna()
df_returns

,INDHOTEL,ORIENTHOT,SAYAJIHOTL,JUBLFOOD,EIHOTEL,SPECIALITY,ROHLTD,WONDERLA,HLVLTD,BLS
date,,,,,,,,,,
2017-08-08,0.006974,-0.025246,0.000000,0.052390,0.014498,0.050569,-0.014429,-0.010508,-0.075789,-0.050119
2017-08-09,-0.029242,-0.027338,-0.002012,-0.023745,-0.012411,-0.030886,-0.028838,0.004073,-0.022779,0.027080
2017-08-10,-0.011098,-0.001479,0.000000,0.014432,-0.000381,-0.030215,-0.033805,-0.013474,-0.034965,-0.025822
2017-08-11,-0.016161,-0.028148,0.000000,-0.016199,0.000762,-0.040120,-0.000473,-0.008959,-0.007246,0.003348
2017-08-14,0.021650,0.027439,0.000000,0.057149,-0.007994,-0.002223,0.022706,0.005483,0.017032,-0.000835
...,...,...,...,...,...,...,...,...,...,...
2022-07-29,-0.001140,-0.027069,0.015540,-0.026881,-0.009502,0.168596,0.011364,0.018640,0.038889,0.015766
2022-08-01,0.028914,0.007949,0.019612,0.017083,-0.011578,0.009101,0.028090,0.022250,0.005348,-0.006623
2022-08-02,-0.007395,0.041798,0.044388,0.011168,0.030120,-0.002122,-0.012942,-0.030716,0.042553,0.004583


In [5]:
# 5-year average annualised return of each stock
# assuming there are 250 trading days in a year

stock_returns = (1 + df_returns.mean()) ** 250 - 1
stock_returns

INDHOTEL      0.281628
ORIENTHOT     0.281471
SAYAJIHOTL    0.099728
JUBLFOOD      0.462523
EIHOTEL       0.132631
SPECIALITY    0.285300
ROHLTD        0.294658
WONDERLA      0.008885
HLVLTD       -0.026207
BLS           0.438118
dtype: float64

In [6]:
# 5-year annualised risk (volatility) of each stock

df_returns.std(ddof=1) * np.sqrt(250)

INDHOTEL      0.403586
ORIENTHOT     0.486158
SAYAJIHOTL    0.453728
JUBLFOOD      0.382512
EIHOTEL       0.421395
SPECIALITY    0.557594
ROHLTD        0.570942
WONDERLA      0.344446
HLVLTD        0.576816
BLS           0.582517
dtype: float64

In [7]:
# Generating a covariance matrix of stock returns

vcv_matrix = df_returns.cov(ddof=1)
vcv_matrix

,INDHOTEL,ORIENTHOT,SAYAJIHOTL,JUBLFOOD,EIHOTEL,SPECIALITY,ROHLTD,WONDERLA,HLVLTD,BLS
INDHOTEL,0.000652,0.000285,0.000053,0.000198,0.000327,0.000255,0.000356,0.000175,0.000209,0.000173
ORIENTHOT,0.000285,0.000945,0.000100,0.000104,0.000302,0.000304,0.000416,0.000139,0.000301,0.000217
SAYAJIHOTL,0.000053,0.000100,0.000823,0.000043,0.000083,0.000161,0.000100,0.000046,0.000125,0.000117
JUBLFOOD,0.000198,0.000104,0.000043,0.000585,0.000173,0.000164,0.000188,0.000119,0.000109,0.000131
EIHOTEL,0.000327,0.000302,0.000083,0.000173,0.000710,0.000275,0.000411,0.000173,0.000253,0.000183
SPECIALITY,0.000255,0.000304,0.000161,0.000164,0.000275,0.001244,0.000399,0.000200,0.000290,0.000294
ROHLTD,0.000356,0.000416,0.000100,0.000188,0.000411,0.000399,0.001304,0.000232,0.000376,0.000282
WONDERLA,0.000175,0.000139,0.000046,0.000119,0.000173,0.000200,0.000232,0.000475,0.000132,0.000183
HLVLTD,0.000209,0.000301,0.000125,0.000109,0.000253,0.000290,0.000376,0.000132,0.001331,0.000285
BLS,0.000173,0.000217,0.000117,0.000131,0.000183,0.000294,0.000282,0.000183,0.000285,0.001357


In [8]:
# Generating an array (or a vector) of equal weights to be used as inputs in the 'minimize' function

num_stocks = len(df.columns)
initial_weights = [1 / num_stocks] * num_stocks
initial_weights

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [9]:
# Developing a function for generating annualised portfolio returns using 'weights' as an input

def getPortReturn(weights):
    port_return = np.dot(np.transpose(weights), stock_returns)
    
    return port_return

In [10]:
# Calculating portfolio return using the initial weights

getPortReturn(initial_weights)

0.2258735153889861

In [11]:
# Developing a function for calculating the portfolio risk using 'weights' as an input

def getPortRisk(weights):
    port_risk = np.sqrt(np.dot(np.transpose(weights), np.dot(vcv_matrix, weights)) * 250)
    
    return port_risk

In [12]:
# Calculating annualised standard deviation of the portfolio (Portfolio risk)

getPortRisk(initial_weights)

0.26606013662556854

In [13]:
# Developing a function for generating a table of returns & risks using 'weights' as an input

def getResultsTable(weights):
    results_df = pd.DataFrame(weights)
    results_df.index = df.columns
    results_df.rename(columns = {results_df.columns[0] : 'weights'}, inplace = True)
    results_df['annualised_returns'] = (1 + df_returns.mean()) ** 250 - 1
    results_df['weighted_returns'] = results_df['weights'] * results_df['annualised_returns']
    results_df['annualised_risks'] = df_returns.std(ddof=1) * np.sqrt(250)
    
    print('Sum of weighted_returns is', round(results_df['weighted_returns'].sum(), 5))
    results_df = round(results_df, 5)
    return results_df

In [14]:
# Turning the results from using initial weights into a Pandas dataframe


getResultsTable(initial_weights)

Sum of weighted_returns is 0.22587


,weights,annualised_returns,weighted_returns,annualised_risks
INDHOTEL,0.1,0.28163,0.02816,0.40359
ORIENTHOT,0.1,0.28147,0.02815,0.48616
SAYAJIHOTL,0.1,0.09973,0.00997,0.45373
JUBLFOOD,0.1,0.46252,0.04625,0.38251
EIHOTEL,0.1,0.13263,0.01326,0.42140
SPECIALITY,0.1,0.28530,0.02853,0.55759
ROHLTD,0.1,0.29466,0.02947,0.57094
WONDERLA,0.1,0.00889,0.00089,0.34445
HLVLTD,0.1,-0.02621,-0.00262,0.57682
BLS,0.1,0.43812,0.04381,0.58252


In [15]:
# Importing market portfolio data (NSE: MICROCAP 250) and calculating it's daily returns

market= pd.read_csv('NSE_MICROCAP250.csv')
market = market[['DateTime', 'Nifty Microcap 250']]
market.rename(columns = {'DateTime' : 'date', 'Nifty Microcap 250' : 'MICROCAP250'}, inplace = True)
market.set_index('date', inplace = True)
market = market.pct_change(1).dropna()
market

,MICROCAP250
date,
08-08-2017 0:00,-0.015044
09-08-2017 0:00,-0.015494
10-08-2017 0:00,-0.038609
11-08-2017 0:00,-0.007618
14-08-2017 0:00,0.025756
...,...
29-07-2022 0:00,0.008704
01-08-2022 0:00,0.015414
02-08-2022 0:00,0.006549


In [16]:
# Annualised return of the market portfolio (MICROCAP 250) for the previous 5 years
market_return =(1 + market['MICROCAP250'].mean()) ** 250 - 1
market_return

0.13539531874890343

In [17]:
# Annualised risk of the market

market_risk = np.std(market['MICROCAP250'], ddof = 1) * np.sqrt(250)
market_risk

0.22410486543096333

In [18]:
# For the 'minimize' function, ensuring the individual weights should be between 0 & 1
# No 'Short' positions allowed

bounds = tuple((0,1) for i in range(num_stocks))
bounds

((0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1))

In [19]:
# Developing a function for comparison of returns

def getResultsReturn(weights):
    print('Return of the portfolio with weights optimised is', getPortReturn(weights))
    print('Return of the market portfolio (MICROCAP 250) is', market_return)
    print('Return of the portfolio with intial weights (equal weights) is', getPortReturn(initial_weights))

In [20]:
# Developing a function for comparison of risks

def getResultsRisk(weights):
    print('Risk of the portfolio with weights optimised is', getPortRisk(weights))
    print('Risk of the market portfolio (MICROCAP 250) is', market_risk)
    print('Risk of the portfolio with intial weights (equal weights) is', getPortRisk(initial_weights))

----------------------------------------------------
## Task 1:

#### Generating optimised weights for the assets in the portfolio where the expected return of the portfolio is equal to the annualised expected return of the market portfolio.

The next lines of codes describe the parameters to be used in the `minimize` function from the *scipy.optimize* package to optimise weights for the portfolio that would result in the **expected return** of the portfolio and market return to be **identical**.  
  
We will input the function *getPortReturn*, *initial_weights* for the initial guess (x0) and bounds of 0 & 1 in the *'minimize'* function.

In [21]:
# Putting in constraints for the 'minimize' function
# Constraint 1 => Sum of all the weights should equal to 1
# Constraint 2 => Portfolio return should be equal to the Market return

cons1 = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
         {'type' : 'eq', 'fun' : lambda x : np.dot(np.transpose(x), stock_returns) - market_return})

In [22]:
# Putting in all the parameters in the 'minimize' function

results1 = minimize(fun = getPortReturn, x0 = initial_weights, bounds = bounds, constraints = cons1)
results1

     fun: 0.13539531865000154
     jac: array([ 0.28162777,  0.28147124,  0.09972784,  0.46252332,  0.13263066,
        0.28530035,  0.29465813,  0.00888517, -0.02620699,  0.43811766])
 message: 'Optimization terminated successfully'
    nfev: 22
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.07988207, 0.07993855, 0.14551742, 0.01460914, 0.13364502,
       0.07855689, 0.0751803 , 0.17829637, 0.19095875, 0.02341549])

In [23]:
# Turning the results from using the optimised weights into a Pandas dataframe

getResultsTable(results1['x'])

Sum of weighted_returns is 0.1354


,weights,annualised_returns,weighted_returns,annualised_risks
INDHOTEL,0.07988,0.28163,0.02250,0.40359
ORIENTHOT,0.07994,0.28147,0.02250,0.48616
SAYAJIHOTL,0.14552,0.09973,0.01451,0.45373
JUBLFOOD,0.01461,0.46252,0.00676,0.38251
EIHOTEL,0.13365,0.13263,0.01773,0.42140
SPECIALITY,0.07856,0.28530,0.02241,0.55759
ROHLTD,0.07518,0.29466,0.02215,0.57094
WONDERLA,0.17830,0.00889,0.00158,0.34445
HLVLTD,0.19096,-0.02621,-0.00500,0.57682
BLS,0.02342,0.43812,0.01026,0.58252


In [24]:
# Comparison of the returns

getResultsReturn(results1['x'])

Return of the portfolio with weights optimised is 0.13539531865000154
Return of the market portfolio (MICROCAP 250) is 0.13539531874890343
Return of the portfolio with intial weights (equal weights) is 0.2258735153889861


**Observation:** The market return and the optimised portfolio's expected return are identical (***13.540%***). Whereas, the expected return would be higher (***22.587%***) if equal weights for the stocks were used instead of the optimised weights.

In [25]:
# Comparison of the risks

getResultsRisk(results1['x'])

Risk of the portfolio with weights optimised is 0.2707559565460811
Risk of the market portfolio (MICROCAP 250) is 0.22410486543096333
Risk of the portfolio with intial weights (equal weights) is 0.26606013662556854


**Observation:** The risk of the market portfolio is the lowest (***22.410%***), compared to the risks of optimised portfolio and  equally-weighted portfolio. The optimised weights used in the portfolio result in a relatively higher risk (***27.076%***) than the equally-weighted portfolio (***26.606%***), *a difference of **47 bps** between them*.

**CONCLUSION:**  
**(1)** For generating a market return of ***13.540%*** over a period of 5 years (5th August, 2017 to 4th August, 2022), the market portfolio (MICROCAP 250) had a standard deviation or volatility (risk) of ***22.410%***. Whereas, for generating the same return, a portfolio of 10 stocks from the *Indian Hospitality sector* would have had a volatility of ***27.076%***, reflecting the **riskier** nature of the portfolio.   

**(2)** Although the market return optimised portfolio and the equally-weighted portfolio would have had a similar level of ***risks*** (***27.076% vs. 26.606%***), the ***returns*** generated vary significantly i.e. ***13.540% & 22.587%*** respectively. This is because the **weights** *ranging from **1.461% to 19.096%** were applied* in the optimised portfolio, where:  
> the ***highest weight*** of **19.096%** was given to the '*HLV Ltd.*' stocks which had a 5-year annualised return of **-2.621%** but had an annualised standard deviation of **57.682%** , and  
> the ***lowest weight*** of **1.461%** was given to the '*Jubilant FoodWorks Ltd.*' stocks which had a 5-year annualised return of **46.252%** but had an annualised standard deviation of **38.251%**.  

These weights significantly bring down the expected portfolio return to the market return level (as programmed). Whereas, the equally-weighted portfolio used a ***single weight*** throughout the whole portfolio (***10.000%***).

--------------------------------------
## Task 2:

#### Generating optimised weights for the assets in the portfolio where the annualised risk of the portfolio is equal to the annualised risk of the market.

The next lines of codes describe the parameters to be used in the `minimize` function from the *scipy.optimize* package to optimise weights for the portfolio that would result in the **annualised risks** of the portfolio and the market to be **identical**.  
  
We will input the function *getPortRisk*,  *initial_weights* for the initial guess (x0), and the bounds of 0 & 1 in the *'minimize'* function here too.

In [26]:
# Putting in constraints for the 'minimize' function
# Constraint 1 => Sum of all the weights should equal to 1
# Constraint 2 => Portfolio risk should be equal to the Market risk.

cons2 = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
         {'type' : 'eq', 'fun' : lambda x : np.sqrt(np.dot(np.transpose(x), np.dot(vcv_matrix, x)) * 250) - market_risk})

In [27]:
# Putting in all the parameters in the 'minimize' function

results2 = minimize(fun = getPortRisk, x0 = initial_weights, bounds = bounds, constraints = cons2)
results2

     fun: 0.23163563001976278
     jac: array([0.23163564, 0.23163563, 0.23163564, 0.23163564, 0.23163564,
       0.23163564, 0.25888861, 0.23163563, 0.23163564, 0.23163563])
 message: 'Positive directional derivative for linesearch'
    nfev: 1026
     nit: 66
    njev: 62
  status: 8
 success: False
       x: array([8.35602448e-02, 7.06912543e-02, 1.99805998e-01, 2.18176513e-01,
       6.09288065e-02, 5.21393677e-03, 5.38621300e-14, 2.72984774e-01,
       4.66619933e-02, 4.19764795e-02])

**Observation:** The 'minimize' function gives out an error message "*Positive directional derivative for linesearch*". **BUT WHY??**  
Explained in Section 3 below.

**CONCLUSION:**  
The current portfolio cannot be optimised to have an annualised risk identical to the benchmark index (*MICROCAP 250*).

-------------------------------------------
## Task 3:

#### Generating optimised weights for the assets in the portfolio where the annualised risk of the portfolio is minimised.

The next lines of codes describe the parameters to be used in the `minimize` function from the *scipy.optimize* package to optimise weights for the portfolio that would result in the **annualised risk** of the portfolio to be **minimised**.  
  
We will input the function *getPortRisk*,  *initial_weights* for the initial guess (x0), and the bounds of 0 & 1 in the *'minimize'* function here too.

In [28]:
# Putting in a constraint for the 'minimize' function
# Constraint => Sum of all the weights should equal to 1

cons3 = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1})

In [29]:
# Putting in all the parameters in the 'minimize' function

results3 = minimize(fun = getPortRisk, x0 = initial_weights, bounds = bounds, constraints = cons3)
results3

     fun: 0.231636160468506
     jac: array([0.23177355, 0.23154981, 0.23142236, 0.23204926, 0.23113988,
       0.23175581, 0.25879102, 0.2315236 , 0.23173119, 0.23168526])
 message: 'Optimization terminated successfully'
    nfev: 66
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([8.40376793e-02, 7.06834711e-02, 1.99562261e-01, 2.18990201e-01,
       5.98454759e-02, 5.37797719e-03, 1.03270257e-17, 2.72684644e-01,
       4.68036231e-02, 4.20146677e-02])

In [30]:
# Turning the results from the optimised weights into a Pandas dataframe

getResultsTable(results3['x'])

Sum of weighted_returns is 0.19383


,weights,annualised_returns,weighted_returns,annualised_risks
INDHOTEL,0.08404,0.28163,0.02367,0.40359
ORIENTHOT,0.07068,0.28147,0.01990,0.48616
SAYAJIHOTL,0.19956,0.09973,0.01990,0.45373
JUBLFOOD,0.21899,0.46252,0.10129,0.38251
EIHOTEL,0.05985,0.13263,0.00794,0.42140
SPECIALITY,0.00538,0.28530,0.00153,0.55759
ROHLTD,0.00000,0.29466,0.00000,0.57094
WONDERLA,0.27268,0.00889,0.00242,0.34445
HLVLTD,0.04680,-0.02621,-0.00123,0.57682
BLS,0.04201,0.43812,0.01841,0.58252


In [31]:
# Comparison of the returns

getResultsReturn(results3['x'])

Return of the portfolio with weights optimised is 0.19382801683284415
Return of the market portfolio (MICROCAP 250) is 0.13539531874890343
Return of the portfolio with intial weights (equal weights) is 0.2258735153889861


**Observation:** The optimised portfolio's expected return (***19.383%***) is higher than the historical return of MICROCAP 250 (***13.540%***). Whereas, the return would have been higher (***22.587%***) if equal weights were used for the stocks in the portfolio instead of the optimised weights.

In [32]:
# Comparison of the risks

getResultsRisk(results3['x'])

Risk of the portfolio with weights optimised is 0.231636160468506
Risk of the market portfolio (MICROCAP 250) is 0.22410486543096333
Risk of the portfolio with intial weights (equal weights) is 0.26606013662556854


**Observation:** The risk of the market portfolio is the lowest (***22.410%***), compared to the risk-optimised portfolio and the equally-weighted portfolio. The `minimize` function generated the optimised weights which have brought the portfolio risk to the lowest possible (***23.164%***), whereas, using the equal-weights for the stocks would have resulted in higher volatility (risk) (***26.606%***).

**In 'Section 2', why did the *scipy.optimize* package's `minimize` function not work?**  
As seen above, after optimisation through the 'minimize' function, the lowest possible risk from the portfolio is *23.164%*. Whereas, the market risk is *22.410%* which is *75.4 bps (0.754%)* lower than 23.164%. Hence, *scipy.optimize* package's 'minimize' function returned an error when asked to bring the portfolio risk to the market risk level in Section 2.

**CONCLUSION:**  
**(1)** An 'Indian Hospitality sector' portfolio of 10 stocks would have had a volatility of ***23.164%*** if the weights of the stocks were risk-optimised, whilst generating ***19.383%*** annualised expected return. Since the portfolio's lowest risk possible is still higher than the market risk of ***22.410%***, this resulted in the 'minimize' function showing an error in Section 2. This shows that the portfolio is still riskier (***75.4 bps***)  than the benchmark index *MICROCAP 250* even at its lowest possible risk.  

**(2)** In comparison to the equally-weighted portfolio, the risk-optimised portfolio carries relatively lower risk (***26.606% vs. 23.164%***), but it would generate relatively lower return (***22.587% vs. 19.383%***), thereby reflecting a direct relationship between returns and risks.  
  
**(3)** To achieve the lowest possible volatility, the risk-optimised portfolio consists of weights ranging from ***0.000% to 27.268%*** where:
> the ***highest weight*** of **27.268%** was applied to the *Wonderla Holidays Ltd.* stocks which had a 5-year annualised return of **0.889%** but had an annualised standard deviation of **34.445%** , and  
> the ***lowest weight*** or 'no allocation' of **0.000%** was given to the '*Royal Orchid Hotels Ltd.*' stocks which had a 5-year annualised return of **29.466%** but had an annualised standard deviation of **57.094%**.

The `minimize` function from *scipy.optimize* package ensured that the portfolio achieved its lowest possible risk, *even if one of the portfolio assets remains unallocated*.

-------------------------------
## Task 4:

#### Invest in the Optimised Portfolio, or the Market Portfolio?

Based on the results from the optimisations, especially from the one in *Section 3*, it would be better to invest in the portfolio  of 10 'Indian Hospitality' stocks than the MICROCAP 250 companies. It has similar risk like the market portfolio, but would generate significantly higher return than the market portfolio.